### CODE FOR SIMULATED ANNEALING

In [226]:
def executeAnnealing(df, m, T, k, alpha, Rw, Rh):
    #output the exact positioning of the rectangles that fit according to first fit layering procedure
    def packTables(establishmentWidth, establishmentHeight, listOfTables):

        #sort list of rectangles in decreasing height,
        listOfTables.sort(reverse=True, key=getSecondElement)
        xSolution = []
        ySolution = []
        tableIndexes = []

        #if set is empty, return empty list
        if not listOfTables:
            return [], [], [], False

        minHeight = 0
        minWidth = 0
        maxHeight = establishmentWidth
        maxWidth = establishmentHeight


        for table in listOfTables:
            #if a table can fit with all currently placed tables
            if ((table[0] + minWidth <= establishmentWidth) and (table[1] >= minHeight and table[1] <= maxHeight)):
                #add table to solutions
                xSolution.append(minWidth)
                ySolution.append(minHeight)
                tableIndexes.append(table[2])

                #set new boundaries for layer
                maxHeight = table[1]
                minWidth = minWidth + table[0]
            #if a table can fit a layer above the previous
            elif(table[1] <= (establishmentHeight-maxHeight)):
                #set new boundaries for layer
                minHeight = maxHeight
                #maxHeight = minHeight + table[0]

                #add table to solutions
                xSolution.append(minWidth)
                ySolution.append(minHeight)
                tableIndexes.append(table[2])

        ifFeasible = False
        if (len(listOfTables) == len(tableIndexes)):
            ifFeasible = True

        return xSolution, ySolution, tableIndexes, ifFeasible

    def getSecondElement(item):
        return item[1]

    def setIncumbent():
        tableConfiguration = []
        for index, row in df.iterrows():
            tableConfiguration.append(row['ifUsed'])
        return tableConfiguration

    #adds or removes 1 random table
    def tableSwap(configuration):
        modifiedConfiguration = configuration
        # generatenumber of unused tables that the removed table will be replaced by
        change = "add" if random.uniform(0, 1) > 0.5 else "remove"

        if (allSame( modifiedConfiguration, 0)):
            change = "add"

        if (allSame( modifiedConfiguration, 1)):
            return modifiedConfiguration

        if (change == "remove"):
            #randomly select a used table to be removed
            while (True):
                randomTableIndex = random.randint(0, len(modifiedConfiguration) - 1)
                if (modifiedConfiguration[randomTableIndex] == 1):
                    modifiedConfiguration[randomTableIndex] = 0
                    break;
        elif (change == "add"):
            while (True):
                randomTableIndex = random.randint(0, len(modifiedConfiguration) - 1)
                if (modifiedConfiguration[randomTableIndex] == 0):
                    modifiedConfiguration[randomTableIndex] = 1
                    break;

        return modifiedConfiguration

    def allSame(configuration, value):
        return all(x == value for x in configuration)

    #checks if a table arrangement is feasible
    def isFeasible(arrangement):

        tempTables = getTableData(arrangement)

        #if a table is missing after packing, it is not feasible
        if(packTables(Rw, Rh, tempTables)[3]):
            return True
        else:
            return False

    def getTableData(arrangement):
        tempTables = []
        for index, row in df.iterrows():
            if(arrangement[index]==1):
                tempTables.append((row['table space width'], row['table space height'], index))
        return tempTables


    #Generate feasible neighbouring solution by packing a subset of all available tables
    def generateFeasibleSolution(currentSolution):
        #print ("generating neighbour of:", currentSolution)
        randomSolution = []

        solutionFound = False
        while(not solutionFound):
            randomSolution = tableSwap(currentSolution[:])
            if(isFeasible(randomSolution)):
                solutionFound = True    

        return randomSolution

    #Check how a feasible solution impacts the objective
    def checkObjective(configuration):
        objective = 0
        index = 0
        for table in configuration:
            if configuration[index]==1:
                objective += df.at[index,'capacity']
            index += 1
        return (objective)

    #Intitialize temperature schedule
    def initializeTemperatureSchedule(T, alpha, k):
        latestTemp = T
        temperatureSchedule = [T]
        for i in range(k-1):
            temperatureSchedule.append(latestTemp*alpha)
            latestTemp = latestTemp*alpha
        return temperatureSchedule

    def acceptCandidate(currentSolutionObjective, candidateSolutionObjective, T):
        #generate random number between 0 and 1
        randomNumber = random.uniform(0, 1)
        acceptanceProbability = math.exp((candidateSolutionObjective - currentSolutionObjective)/T)

        #if candidate solution is better, then accept it
        if (candidateSolutionObjective > currentSolutionObjective):
            return 'betterCandidateAccepted'
        elif (randomNumber <= acceptanceProbability):
            return 'worseCandidateAccepted'
        else:
            return 'worseCandidateRejected'
    
    def displaySolutionData(solution, solutionObjective, ifIncumbent):
        if (ifIncumbent):
            #print('Final Incumbent Solution: ', solution)
            print('Final Incumbent Objective Solution: ', solutionObjective)
            #print('Final Incumbent x Values: ', packTables(Rw, Rh, getTableData(solution))[0])
            #print('Final Incumbent y Values: ', packTables(Rw, Rh, getTableData(solution))[1])
            
            d = {'ifUsed' : solution, 'x-coord' : packTables(Rw, Rh, getTableData(solution))[0], 'y-coord' : packTables(Rw, Rh, getTableData(solution))[1], 'capacity' : instanceData[3], 'table space width' : instanceData[4], 'table space height' : instanceData[5]}
            dfOptimized = pd.DataFrame(data=dataSolved)
            print(dfOptimized)
        else:
            print('Neighbouring Solution: ',solution)
            print('Neighbouring Solution Objective: ', solutionObjective)
            print('neighbouring Solution x Values: ', packTables(Rw, Rh, getTableData(solution))[0])
            print('neighbouring Solution y Values: ', packTables(Rw, Rh, getTableData(solution))[1])

    
    
    print("---ANNEALING STARTED---")
        #construct initial incumbent
    incumbent = setIncumbent()
    incumbentObjective = checkObjective(incumbent)
        
    #initialize current solution to be the incumbent
    currentSolution = incumbent
    currentSolutionObjective = checkObjective(incumbent)

    #initializeTemperatureSchedule
    temperatureSchedule =  initializeTemperatureSchedule(T, alpha, k)
    print('Temperature schedule: ', temperatureSchedule)
    

    for temperature in temperatureSchedule:
        for index in range(m):
            #print('Room Width:', Rw)
            #print('Room Height:', Rh)
            #print ('Temperature: ', temperature)
            #print("INCUMBENT: ", incumbent)
            #print("INCUMBENT OBJECTIVE: ", incumbentObjective)
            #print('CURRENT SOLUTION: ',currentSolution)
            #print('CURRENT SOLUTION OBJECTIVE: ',currentSolutionObjective)
            
            #generate a candidate solution
            candidateSolution = generateFeasibleSolution(currentSolution)
            candidateSolutionObjective = checkObjective(candidateSolution)
            
            #display candidate data
            #displaySolutionData(candidateSolution, candidateSolutionObjective, False)
    
            #accept candidate solution to be current solution with some degree of probability
            acceptanceResult = acceptCandidate(currentSolutionObjective, candidateSolutionObjective, temperature)
            #print(acceptanceResult, '\n')
            
            #adjust current solution if candidate is better or if adjust current solution if it is worse with some probability
            if (acceptanceResult == 'betterCandidateAccepted' or acceptanceResult == 'worseCandidateAccepted' ):
                currentSolution = candidateSolution
                currentSolutionObjective = candidateSolutionObjective

            #only replace incumbent if the candidate is better than it
            if(currentSolutionObjective > incumbentObjective):
                print("INCUMBENT OBJ CHANGE FROM:", incumbentObjective, "to", currentSolutionObjective)
                #print("INCUMBENT CHANGE FROM:", incumbent, "to", currentSolution)
                incumbent = currentSolution
                incumbentObjective = currentSolutionObjective
    
    displaySolutionData(incumbent, incumbentObjective, True)
    
    print("---ANNEALING ENDED---")

### APPLY CONSTRUCTION HEURISTIC THEN SIMULATED ANNEALING

In [227]:
import json 
import pprint
import pandas as pd
import random
import math
import time

pp = pprint.PrettyPrinter(indent=2)

with open('feasible_solutions.json') as f:
  test_loaded_instances = json.load(f)

#pp.pprint(test_loaded_instances)


def loadList(establishmentList, ifOriginal):
    ifUsed = []
    x_coord = []
    y_coord =[]
    capacity = []
    tableWidth = []
    tableHeight = []
    
    for table in establishmentList:
        if (ifOriginal):
            ifUsed.append(0)
            x_coord.append(0)
            y_coord.append(0)
        else:
            ifUsed.append(0)
            x_coord.append(0)
            y_coord.append(0)
        capacity.append(table["capacity"])
        tableWidth.append(table["width"])
        tableHeight.append(table["height"])
        
    return ifUsed, x_coord, y_coord, capacity, tableWidth, tableHeight

def loadSolutionList(establishmentList, originalTablesList, originalTablesX, originalTablesY):
    for t in establishmentList:
        originalTablesList[t["original_index"]] = 1
        originalTablesX[t["original_index"]] = t["x_coord"]
        originalTablesY[t["original_index"]] = t["y_coord"]
              
    return originalTablesList, originalTablesX, originalTablesY
  
def getObjective(configuration, dataFrame):
        objective = 0
        index = 0
        for table in configuration:
            if configuration[index]==1:
                objective += dataFrame.at[index,'capacity']
            index += 1
        return (objective)
    
#Load all instances and initial solutions into dataframe
for index, instance in enumerate(test_loaded_instances):
    #load original instance data in dataframe
    print("ORIGINAL LIST FOR INSTANCE: ", index)
    instanceData = loadList(instance["original_list"], True)
    dataOriginal = {'ifUsed' : instanceData[0], 'x-coord' : instanceData[1], 'y-coord' : instanceData[2], 'capacity' : instanceData[3], 'table space width' : instanceData[4], 'table space height' : instanceData[5]}
    dfOriginal = pd.DataFrame(data=dataOriginal)
    print(dfOriginal, "\n")
    
    #load instance data after solving into dataframe
    print("---CONSTRUCTION STARTED---")
    print("Solution List after Construction Heuristic for Instance: ", index)
    instanceDataSolved = loadSolutionList(instance["solution_list"], instanceData[0][:], instanceData[1][:], instanceData[2][:])
    dataSolved = {'ifUsed' : instanceDataSolved[0], 'x-coord' : instanceDataSolved[1], 'y-coord' : instanceDataSolved[2], 'capacity' : instanceData[3], 'table space width' : instanceData[4], 'table space height' : instanceData[5]}
    dfSolved = pd.DataFrame(data=dataSolved)
    print("Objective Value Before Annealing: ", getObjective(instanceDataSolved[0], dfSolved))
    print(dfSolved, "\n")
    print("---CONSTRUCTION ENDED---")
    
    #conduct simulated annealing with predetermined annealing parameters
    start_time = time.time()
    #executeAnnealing(dfSolved, 2, 100, 4, 0.2, instance['original_instance']['width'], instance['original_instance']['height'])
    
    executeAnnealing(dfSolved, 5, 200, 8, 0.3, instance['original_instance']['width'], instance['original_instance']['height'])
    print("COMPUTATIONAL TIME FOR INSTANCE:",index,"WAS", str(time.time() - start_time) + ' seconds', '\n')

ORIGINAL LIST FOR INSTANCE:  0
    ifUsed  x-coord  y-coord  capacity  table space width  table space height
0        0        0        0        15                  5                   5
1        0        0        0        15                  5                   5
2        0        0        0        15                  5                   5
3        0        0        0        12                  4                   4
4        0        0        0        12                  4                   4
5        0        0        0        12                  4                   4
6        0        0        0        10                  3                   3
7        0        0        0        10                  3                   3
8        0        0        0        10                  3                   3
9        0        0        0        12                  2                   2
10       0        0        0        12                  2                   2 

---CONSTRUCTION STARTED---
Solu

INCUMBENT OBJ CHANGE FROM: 396 to 406
INCUMBENT OBJ CHANGE FROM: 406 to 408
INCUMBENT OBJ CHANGE FROM: 408 to 418
INCUMBENT OBJ CHANGE FROM: 418 to 430
INCUMBENT OBJ CHANGE FROM: 430 to 432
INCUMBENT OBJ CHANGE FROM: 432 to 444
INCUMBENT OBJ CHANGE FROM: 444 to 454
INCUMBENT OBJ CHANGE FROM: 454 to 464
INCUMBENT OBJ CHANGE FROM: 464 to 476
INCUMBENT OBJ CHANGE FROM: 476 to 486
INCUMBENT OBJ CHANGE FROM: 486 to 498
INCUMBENT OBJ CHANGE FROM: 498 to 510
INCUMBENT OBJ CHANGE FROM: 510 to 522
INCUMBENT OBJ CHANGE FROM: 522 to 532
INCUMBENT OBJ CHANGE FROM: 532 to 547
INCUMBENT OBJ CHANGE FROM: 547 to 559
INCUMBENT OBJ CHANGE FROM: 559 to 571
INCUMBENT OBJ CHANGE FROM: 571 to 581
INCUMBENT OBJ CHANGE FROM: 581 to 593
Final Incumbent Objective Solution:  593
    ifUsed  x-coord  y-coord  capacity  table space width  table space height
0        1        0        0        90                 20                  20
1        1       20        0        15                 15                  15
2  

### COMPUTATIONAL TIME